In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from format_data import read_dataset, aggregate_dataset, clean_dataset, sliding_window

In [4]:
dataset = pd.read_csv('data/dataset_mood_smartphone.csv')
dataset = read_dataset(dataset)
dataset = aggregate_dataset(dataset)
dataset = clean_dataset(dataset)
samples = sliding_window(dataset)

/Users/davidvos/Developer/DMT-Assignment1/format_data.py:42: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  dataset = dataset.fillna(dataset.mean())


In [50]:
# Taking averages over 5 days and format data
X = []
y = []

for index, sample in enumerate(samples):
    week_average = np.average(sample[0], axis=0)
    X.append(week_average)
    y.append(sample[1])
    
X = np.array(X)
y = np.array(y)
print(np.shape(X))
print(np.shape(y))

(830, 7)
(830,)


In [47]:
# Testing fake data to see if model makes sense

# X = [np.array([1]) for x in range(100)]
# X += [np.array([2]) for y in range(100)]
# X = np.array(X)

# y = [1 for x in range(100)]
# y += [2 for y in range(100)]
# y = np.array(y)

X_train = X[:500]  
X_test = X[500:]   

y_train = y[:500]  
y_test = y[500:]

In [ ]:
# Run simple MLP classifier and calculate accuracy
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(X_train, y_train)

predictions = clf.predict(X_test)

correct = 0
for index, prediction in enumerate(predictions):
    if prediction == y_test[index]:
        correct += 1
print(f'Accuracy of model is {correct/len(predictions)}')

In [49]:
correct = 0
for index, prediction in enumerate(predictions):
    if prediction == y_test[index]:
        correct += 1
print(f'Accuracy of model is {correct/len(predictions)}')

Accuracy of model is 0.4696969696969697


In [62]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable

class LSTM(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
#         self.seq_length = seq_length
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)
        
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        
        c_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        
        # Propagate input through LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        
        h_out = h_out.view(-1, self.hidden_size)
        
        out = self.fc(h_out)
        
        return out

In [63]:
trainX = Variable(torch.Tensor(X_train.astype(float)))
trainY = Variable(torch.Tensor(y_train.astype(float)))

testX = Variable(torch.Tensor(np.array(X_test.astype(float))))
testY = Variable(torch.Tensor(np.array(y_test.astype(float))))

num_epochs = 2000
learning_rate = 0.01

input_size = 1
hidden_size = 2
num_layers = 1

num_classes = 1

lstm = LSTM(num_classes, input_size, hidden_size, num_layers)

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)
#optimizer = torch.optim.SGD(lstm.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    outputs = lstm(trainX)
    optimizer.zero_grad()
    
    # obtain the loss function
    loss = criterion(outputs, trainY)
    
    loss.backward()
    
    optimizer.step()
    if epoch % 100 == 0:
        print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))

RuntimeError: For unbatched 2-D input, hx and cx should also be 2-D but got (3-D, 3-D) tensors

In [ ]:
model = LSTMClassification(input_.shape[2], hidden_dim=6, target_size=1)